# --- Day 3: Spiral Memory ---
You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

17  16  15  14  13<br>
18   5   4   3  12<br>
19   6   1   2  11<br>
20   7   8   9  10<br>
21  22  23---> ...<br>
While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

Data from square 1 is carried 0 steps, since it's at the access port.
Data from square 12 is carried 3 steps, such as: down, left, left.
Data from square 23 is carried only 2 steps: up twice.
Data from square 1024 must be carried 31 steps.
How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

In [50]:
nr = 312051

In [3]:
import numpy as np

In [49]:
np.arange(9)*2*4

array([ 0,  8, 16, 24, 32, 40, 48, 56, 64])

In [59]:
# calculate number of squares up until this ring:
def nos(ring):
    return np.sum(np.arange(ring)*8)+1

In [74]:
def determinering(nr):
    ring, sm = 0, 0
    while sm < nr:
        ring += 1
        sm = nos(ring)
    return ring - 1 # because when the loop stops it has already crossed the condition
print('square {} is on ring {}'.format(nr, determinering(nr)))

square 312051 is on ring 279


Geometry thoughts...
* a ring is devided into 8 stretches, each stretch has length 'ring'
* uneven stretches run from (1-)cornerpoints towards the middle, even stretches from the middle towards the cornerpoints
* at the middle the minimum distance == ring number
* at the cornerpoint the distance is twice the ring number

--> next steps: determine if it is on an odd or even stretch, calculate the remainder and then the distance

In [78]:
def determinedist(nr):
    ring = determinering(nr)
    stretch = np.floor_divide(nr-nos(ring),ring)+1
    remainder = (nr-nos(ring)) % ring
    if stretch%2 == 0:
        distance = ring + remainder
    else:
        distance = 2*ring - remainder
    return distance
    

print('the distance of square {} to square 1 = {}'.format(1024, determinedist(1024)))

the distance of square 1024 to square 1 = 31


## --- Part Two ---
As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

Square 1 starts with the value 1.
Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.
Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
What is the first value written that is larger than your puzzle input?




(0,1)
(-1,1)
(-1,0)
(-1,-1)
(0,-1)
(1,-1)
(1,0)
(1,1)

(1,2)
(0,2)
(-1,2)
(-2,2)
(-2,1)
(-2,0)
(-2,-1)


In [31]:
def ringcoords(ring):
    i = ring-1
    j = ring
    coords = [(i,j)]
    while -i < j:
        i -= 1
        coords.append((i,j))
    while j > i:
        j -= 1
        coords.append((i,j))
    while i < -j:
        i += 1
        coords.append((i,j))
    while j < i:
        j += 1
        coords.append((i,j))
    return coords

In [36]:
def surrounding(coords):
    i, j = coords
    surrounding = []
    for point in ringcoords(1):
        a, b = point
        surrounding.append((a+i, b+j))
    return surrounding

In [47]:
def find_next_largest(nr):
    squares = {(0,0): 1}
    ring = 1
    while ring < nr:
        coords = ringcoords(ring)
        for coord in coords:
            sur = surrounding(coord)
            value = 0
            for s in sur:
                if s in squares.keys():
                    value += squares[s]
            if value > nr:
                return value
            squares[coord] = value
        ring += 1
    return squares

In [52]:
find_next_largest(nr)

312453